In [1]:
%%javascript
/**********************************************************************************************
Known Mathjax Issue with Chrome - a rounding issue adds a border to the right of mathjax markup
https://github.com/mathjax/MathJax/issues/1300
A quick hack to fix this based on stackoverflow discussions: 
http://stackoverflow.com/questions/34277967/chrome-rendering-mathjax-equations-with-a-trailing-vertical-line
**********************************************************************************************/

$('.math>span').css("border-left-color","transparent")

<IPython.core.display.Javascript object>

In [ ]:
%reload_ext autoreload
%autoreload 2

# DATASCI W261 - Machine Learning At Scale
## Assignment - Week 09
---

* **Name:**  Megan Jasek
* **Email:**  meganjasek@ischool.berkeley.edu
* **Class Name:**  W261-2
* **Date:**  7/17/16

---
### Instructions

Due by 07/17/2016

[Submission Link - Google Form](https://docs.google.com/forms/d/1ZOr9RnIe_A06AcZDB6K1mJN4vrLeSmS2PD6Xm3eOiis/viewform?usp=send_form) 

### Documents:
* IPython Notebook, published and viewable online.
* PDF export of IPython Notebook.
    
### Useful References

* Data-intensive text processing with MapReduce. San Rafael, CA: Morgan & Claypool Publishers. Chapter 5. 


---

<h2 style="color:darkblue">HW 9 Dataset</h2>

Note that all referenced files life in the enclosing directory. [Checkout the Data subdirectory on Dropbox](https://www.dropbox.com/sh/2c0k5adwz36lkcw/AAAAKsjQfF9uHfv-X9mCqr9wa?dl=0) or the AWS S3 buckets (details contained each question). 

<h2 style="color:darkblue"> HW 9.0: Short answer questions </h2>

__ What is PageRank and what is it used for in the context of web search?__



<hr>

__ What modifications have to be made to the webgraph in order to leverage the machinery of Markov Chains to compute the Steady State Distibution? __



<hr>

__ OPTIONAL: In topic-specific pagerank, how can we ensure that the irreducible property is satifsied? (HINT: see HW9.4) __



<hr>


<h2 style="color:darkblue"> HW 9.1: MRJob implementation of basic PageRank </h2>

Write a basic MRJob implementation of the iterative PageRank algorithm that takes sparse adjacency lists as input (as explored in HW 7).

Make sure that you implementation utilizes teleportation (1-damping/the number of nodes in the network), and further, distributes the mass of dangling nodes with each iteration so that the output of each iteration is correctly normalized (sums to 1).


[NOTE: The PageRank algorithm assumes that a random surfer (walker), starting from a random web page, chooses the next page to which it will move by clicking at random, with probability d,one of the hyperlinks in the current page. This probability is represented by a so-called *damping factor* d, where d ∈ (0, 1). Otherwise, with probability (1 − d), the surfer jumps to any web page in the network. If a page is a dangling end, meaning it has no outgoing hyperlinks, the random surfer selects an arbitrary web page from a uniform distribution and “teleports” to that page]


As you build your code, use the test data:

> s3://ucb-mids-mls-networks/PageRank-test.txt

Or under the Data Subfolder for HW7 on Dropbox with the same file name. 
> Dropbox: https://www.dropbox.com/sh/2c0k5adwz36lkcw/AAAAKsjQfF9uHfv-X9mCqr9wa?dl=0

with teleportation parameter set to 0.15 (1-d, where d, the damping factor is set to 0.85), and crosscheck your work with the true result, displayed in the first image in the [Wikipedia article](https://en.wikipedia.org/wiki/PageRank)
and here for reference are the corresponding PageRank probabilities:
<pre>

A, 0.033
B, 0.384
C, 0.343
D, 0.039
E, 0.081
F, 0.039
G, 0.016
H, 0.016
I, 0.016
J, 0.016
K, 0.016

</pre>

<h2 style="color:darkgreen"> HW 9.1 Implementation </h2>

### Create test file:  PageRank-test.txt

In [3]:
%%writefile PageRank-test.txt
B	{'C': 1}
C	{'B': 1}
D	{'A': 1, 'B': 1}
E	{'D': 1, 'B': 1, 'F': 1}
F	{'B': 1, 'E': 1}
G	{'B': 1, 'E': 1}
H	{'B': 1, 'E': 1}
I	{'B': 1, 'E': 1}
J	{'E': 1}
K	{'E': 1}

Writing PageRank-test.txt


### Expected Output from PageRank-test.txt

https://en.wikipedia.org/wiki/PageRank

and here for reference are the corresponding PageRank probabilities:

A, 0.033  
B, 0.384  
C, 0.343  
D, 0.039  
E, 0.081  
F, 0.039  
G, 0.016  
H, 0.016  
I, 0.016  
J, 0.016  
K, 0.016  

0.999 = 0.033+0.384+0.343+0.039+0.081+0.039+0.016+0.016+0.016+0.016+0.016

### Create test file:  Lecture9_10_test.txt

In [2]:
%%writefile Lecture9_10_test.txt
N1	{'N2': 1, 'N4': 1}
N2	{'N3': 1, 'N5': 1}
N3	{'N4': 1}
N4	{'N5': 1}
N5	{'N1': 1, 'N2': 1, 'N3': 1}

Writing Lecture9_10_test.txt


### Expected Output from Lecture9_10_test.txt

Iteration 10  
N1 [{u'N2': 1, u'N4': 1}, 0.10593707133058988]  
N2 [{u'N3': 1, u'N5': 1}, 0.1575895919067216]  
N3 [{u'N4': 1}, 0.18432891803840884]  
N4 [{u'N5': 1}, 0.23511445473251036]  
N5 [{u'N1': 1, u'N2': 1, u'N3': 1}, 0.31702996399176964]  
Total Error:  0.00860982510288  

### Create test file:  Lecture9_10d_test.txt

In [46]:
%%writefile Lecture9_10d_test.txt
N1	{'N2': 1, 'N4': 1}
N2	{'N3': 1, 'N5': 1}
N3	{'N4': 1}
N4	{'N5': 1, 'N6': 1}
N5	{'N1': 1, 'N2': 1, 'N3': 1}

Writing Lecture9_10d_test.txt


In [1]:
%%writefile PageRankInit.py
from mrjob.job import MRJob
from mrjob.step import MRStep
import ast
import json

# This class takes an adjacency list as input and outputs and initialized work file with 
# the following format:
# node_number \t [adjacency_list, infinity, '', 'U']
# One starting node is defined as an argument passed in to the class.  The default is node '1'.
# That starting node has the format:
# node_number \t [adjacency_list, 0.0, '', 'Q']
# U means Unvisited
# Q means Queued in the queue
class MRPageRankInit(MRJob):
    def __init__(self, *args, **kwargs):
        super(MRPageRankInit, self).__init__(*args, **kwargs)
        self.total_nodes_calculated = False
        self.total_nodes = 0

    # For the node equal to the start index, yield
    #    node_number \t tuple(adj_list, 0.0, "", 'Q')
    # For all other nodes, yield
    #    node_number \t tuple(adj_list, infinity, "", 'U')    
    def mapper(self, _, line):
        node_num, adj_dict = line.strip().split('\t')
        adj_dict = ast.literal_eval(adj_dict)
        self.total_nodes += 1
        yield node_num, tuple((adj_dict, 1.0))
        yield '*' + node_num, 1
        for node in adj_dict:
            yield node, tuple(({}, 1.0))
            yield '*' + node, 1

    def reducer(self, node_num, data):
        if node_num[0] == '*':
            self.total_nodes += 1
        else:
            if self.total_nodes_calculated == False:
                yield '*total_nodes', self.total_nodes
                self.total_nodes_calculated = True
            f_adj_dict = {}
            for adj_dict, value in data:
                if adj_dict != {} and f_adj_dict == {}:
                    f_adj_dict = adj_dict
            yield node_num, tuple((f_adj_dict, 1.0/self.total_nodes))
        
    # Create the steps for the MRJob.  There is only a mapper in this job.
    def steps(self):
        JOBCONF_STEP = {
            'mapreduce.job.reduces': '1'
        }
        return [
            MRStep(jobconf=JOBCONF_STEP,
                   mapper=self.mapper,
                   reducer=self.reducer)
               ]
            
if __name__ == '__main__':
    MRPageRankInit.run()

Overwriting PageRankInit.py


In [3]:
#!python PageRankInit.py Lecture9_10_test.txt
!python PageRankInit.py PageRank-test.txt
#!python PageRankInit.py Lecture9_10d_test.txt

No configs found; falling back on auto-configuration
Creating temp directory /tmp/PageRankInit.hadoop.20160713.211954.998358
Running step 1 of 1...
Streaming final output from /tmp/PageRankInit.hadoop.20160713.211954.998358/output...
"*total_nodes"	11
"A"	[{}, 0.09090909090909091]
"B"	[{"C": 1}, 0.09090909090909091]
"C"	[{"B": 1}, 0.09090909090909091]
"D"	[{"A": 1, "B": 1}, 0.09090909090909091]
"E"	[{"B": 1, "D": 1, "F": 1}, 0.09090909090909091]
"F"	[{"B": 1, "E": 1}, 0.09090909090909091]
"G"	[{"B": 1, "E": 1}, 0.09090909090909091]
"H"	[{"B": 1, "E": 1}, 0.09090909090909091]
"I"	[{"B": 1, "E": 1}, 0.09090909090909091]
"J"	[{"E": 1}, 0.09090909090909091]
"K"	[{"E": 1}, 0.09090909090909091]
Removing temp directory /tmp/PageRankInit.hadoop.20160713.211954.998358...


In [1]:
%%writefile PageRank.py
from mrjob.job import MRJob
from mrjob.step import MRStep
from collections import Counter
import ast
import json

# This class implements the Single Source Shortest Path (SSSP) breadth-first search
# algorithm for an unweighted graph in MapReduce.  The functions of the mapper and reducer
# are explained below.  Once a node is put in to the V state its shortest path has been found
# (this is not true for a weighted graph)
class MRPageRank(MRJob):
    def configure_options(self):
        # Configure a new command line option to capture the stop_index for the shortest path
        super(MRPageRank, self).configure_options()
        self.add_passthrough_option('--damping_factor', type='float', default=0.15)
        self.add_passthrough_option('--total_nodes', type='float', default=1.0)
    
    def __init__(self, *args, **kwargs):
        super(MRPageRank, self).__init__(*args, **kwargs)
        self.damping_factor = self.options.damping_factor
        self.total_nodes = self.options.total_nodes
        self.total_page_rank_dangling = 0.0
        self.total_page_rank_outgoing = 0.0
        self.total_page_rank_missing = 0.0
    
    def mapper(self, _, line):
        # If a node is in the Q state then yield 2 things.  1.  The adjacency list, the
        # minimum distance and the shortest path plus its own node name as this is the shortest
        # path that will be found.  2.  All of the nodes in its adj_list each with the following:
        # An empty adjacency list, the current min_dist plus the distance to this node, the
        # current shortest path plus the path to this node, state of Q.
        node_num, data = line.strip().split('\t')
        data = json.loads(data)
        node_num = node_num.strip('"')
        adj_dict = data[0]
        degree = float(len(adj_dict))
        page_rank = data[1]
        # Yield the node to preserve the graph structure
        if adj_dict == {}:
            #print node_num, tuple((adj_dict, page_rank))
            #yield node_num, tuple((adj_dict, page_rank))
            yield node_num, tuple((adj_dict, 0.0))
        else:
            #print node_num, tuple((adj_dict, 0.0))
            yield node_num, tuple((adj_dict, 0.0))
        for node, value in adj_dict.iteritems():
            #print node, tuple(({}, page_rank/degree))        
            yield node, tuple(({}, page_rank/degree))
    
    # For each element in the data list, do the following:
    # - Use the adjacency list for a node that is not {}, there will always be one.
    # - Find the minimum distance of all of the elements of the list, use the shortest
    # path and state from this element as well.
    # If an element is in the Q state and there is no_stop_index, then increment the
    # 'Number_In_Q' counter
    # If an element is in the V state (meaning its shortest path has already been found) and
    # it is the stopping index, then increment the 'Stop_Index_Found' counter.
    def reducer(self, node, data):
        f_adj_dict = Counter()
        f_page_rank = 0.0
        for adj_dict, page_rank in data:
            #print adj_dict, page_rank
            if adj_dict != {} and f_adj_dict == {}:
                f_adj_dict = adj_dict
            f_page_rank += float(page_rank)
        if f_adj_dict == {}:
            print 'd page_rank', node, f_page_rank
            self.total_page_rank_dangling += f_page_rank
        else:
            print 'out page_rank', node, f_page_rank
            self.total_page_rank_outgoing += f_page_rank
        yield node, tuple((f_adj_dict, f_page_rank))
    
    def reducer_final(self):
        print '*total_dangling', self.total_page_rank_dangling
        yield '*total_dangling', self.total_page_rank_dangling
        print '*total_outgoing', self.total_page_rank_outgoing
        print '*total', self.total_page_rank_outgoing+self.total_page_rank_dangling
        yield '*total_outgoing', self.total_page_rank_outgoing
        self.total_page_rank_missing = 1-(self.total_page_rank_outgoing+self.total_page_rank_dangling)
        print '*total_missing', self.total_page_rank_missing
        yield '*total_missing', self.total_page_rank_missing
        
    def reducer_dangling_nodes(self, node, data):
        if node == '*total_dangling':
            for d in data:
                self.total_page_rank_dangling += float(d)
            #print self.total_page_rank_dangling
        elif node == '*total_outgoing':
            for d in data:
                self.total_page_rank_outgoing += float(d)
            #print self.total_page_rank_dangling
        elif node == '*total_missing':
            for d in data:
                self.total_page_rank_missing += float(d)
            #print self.total_page_rank_dangling
        else:
            for adj_dict, page_rank in data:
                #print adj_dict, page_rank
                #print self.total_page_rank_dangling
                #print self.total_page_rank_outgoing
                #print self.total_page_rank_missing
                #f_page_rank = self.damping_factor*(1.0/self.total_nodes)+ \
                #    (1-self.damping_factor)*((self.total_page_rank_dangling/self.total_nodes)+page_rank)
                f_page_rank = (1-self.damping_factor)*(1.0/self.total_nodes)+ \
                    self.damping_factor*((self.total_page_rank_missing/self.total_nodes)+page_rank)
                yield node, tuple((adj_dict, f_page_rank))

    def steps(self):
        JOBCONF_STEP = {
            'mapreduce.job.reduces': '1'
        }
        return [
            MRStep(jobconf=JOBCONF_STEP,
                   mapper=self.mapper,
                   reducer=self.reducer, reducer_final=self.reducer_final),
            MRStep(jobconf=JOBCONF_STEP,
                   reducer=self.reducer_dangling_nodes)
               ]
            
if __name__ == '__main__':
    MRPageRank.run()

Overwriting PageRank.py


In [20]:
def stop_criterion_reached(old_page_ranks, page_ranks, epsilon):
    Stop = False
    total_error = 0.0
    for pr1, pr2 in zip(old_page_ranks, page_ranks):
        total_error += abs(pr1-pr2)
    print 'Total Error: ', total_error
    if total_error < epsilon:
        Stop = True
    return Stop

In [21]:
#%reload_ext autoreload
#%autoreload 2
#from PageRank import MRPageRank
#from PageRankInit import MRPageRankInit
import PageRank
import PageRankInit
reload(PageRank)
reload(PageRankInit)
import json

damping_factor = 0.85
#damping_factor = 1.0
epsilon = 0.01

# Set the name of the file that gets passed from iteration to iteration
work_filename = 'work_table.txt'

# Initialize the work table file
page_ranks = []
#mr_job = PageRankInit.MRPageRankInit(args=['Lecture9_10_test.txt'])
#mr_job = PageRankInit.MRPageRankInit(args=['Lecture9_10d_test.txt'])
mr_job = PageRankInit.MRPageRankInit(args=['PageRank-test.txt'])
with mr_job.make_runner() as runner, open(work_filename, 'w') as f: 
    runner.run()
    # stream_output: get access of the output 
    for line in runner.stream_output():
        key, value =  mr_job.parse_output_line(line)
        if key == '*total_nodes':
            total_nodes = float(value)
        else:
            page_ranks.append(value[1])
            f.write(key+'\t'+json.dumps(value)+'\n')

# Run the SSSP MRJob
mr_job = PageRank.MRPageRank(args=[work_filename, '--damping_factor', damping_factor, '--total_nodes', total_nodes])
    
# Update work table file iteratively
i = 0
# Set stop condition to False
Stop = False
#while(Stop == False):
while(i < 40):
    work_table = {}
    old_page_ranks = page_ranks
    page_ranks = []
    # Print the iteration number
    print('Iteration %d' % (i))
    with mr_job.make_runner() as runner: 
        runner.run()
        # stream_output: get access of the output 
        for line in runner.stream_output():
            key,value =  mr_job.parse_output_line(line)
            print key, value
            work_table[key] = value
            page_ranks.append(value[1])
        
        # Update work_table for the next iteration
        with open(work_filename, 'w') as f:
            for key, value in work_table.iteritems():
                f.write(key+'\t'+json.dumps(value)+'\n')
        
        Stop = stop_criterion_reached(old_page_ranks, page_ranks, epsilon)
        print('Sum of page ranks: %f' % (sum(page_ranks)))
    i += 1

Iteration 0
d page_rank A 0.0454545454545
out page_rank B 0.348484848485
out page_rank C 0.0909090909091
out page_rank D 0.030303030303
out page_rank E 0.363636363636
out page_rank F 0.030303030303
out page_rank G 0.0
out page_rank H 0.0
out page_rank I 0.0
out page_rank J 0.0
out page_rank K 0.0
*total_dangling 0.0454545454545
*total_outgoing 0.863636363636
*total 0.909090909091
*total_missing 0.0909090909091
A [{}, 0.059297520661157024]
B [{u'C': 1}, 0.31687327823691464]
C [{u'B': 1}, 0.09793388429752066]
D [{u'A': 1, u'B': 1}, 0.046418732782369146]
E [{u'B': 1, u'D': 1, u'F': 1}, 0.32975206611570246]
F [{u'B': 1, u'E': 1}, 0.046418732782369146]
G [{u'B': 1, u'E': 1}, 0.02066115702479339]
H [{u'B': 1, u'E': 1}, 0.02066115702479339]
I [{u'B': 1, u'E': 1}, 0.02066115702479339]
J [{u'E': 1}, 0.02066115702479339]
K [{u'E': 1}, 0.02066115702479339]
Total Error:  0.943663911846
Sum of page ranks: 1.000000
Iteration 1
d page_rank A 0.0232093663912
out page_rank B 0.285261707989
out page_ran

In [4]:
print(0.136363636364/11)
print(0.136363636364/11 + 1/11.0)

0.0123966942149
0.103305785124


In [75]:
print(0.10530158257403283+0.1579039788294203+0.18422267372257844+0.23678682235503043+0.3157849425189383)

1.0


In [2]:
## Drivers & Runners

In [3]:
## Run Scripts, S3 Sync

<h2 style="color:darkgreen">  HW 9.1 Analysis </h2>




<br><br>

<h2 style="color:darkblue"> HW 9.2: Exploring PageRank teleportation and network plots </h2>

* In order to overcome  problems such as disconnected components, the damping factor (a typical value for d is 0.85) can be varied. 
* Using the graph in HW1, plot the test graph (using networkx, https://networkx.github.io/) for several values of the damping parameter alpha, so that each nodes radius is proportional to its PageRank score. 
* In particular you should do this for the following damping factors: [0,0.25,0.5,0.75, 0.85, 1]. 
* Note your plots should look like the following: https://en.wikipedia.org/wiki/PageRank#/media/File:PageRanks-Example.svg

<h2 style="color:darkgreen"> HW 9.2 Implementation </h2>

In [1]:
## Code goes here

In [2]:
## Drivers & Runners

In [3]:
## Run Scripts, S3 Sync

<h2 style="color:darkgreen">  HW 9.2 Analysis </h2>




<br><br>

<h2 style="color:darkblue"> HW 9.3: Applying PageRank to the Wikipedia hyperlinks network </h2>

* Run your PageRank implementation on the Wikipedia dataset for 5 iterations, and display the top 100 ranked nodes (with alpha = 0.85).
* Run your PageRank implementation on the Wikipedia dataset for 10 iterations, and display the top 100 ranked nodes (with teleportation factor of 0.15).
* Have the top 100 ranked pages changed? Comment on your findings. 
* Plot the pagerank values for the top 100 pages resulting from the 5 iterations run. Then plot the pagerank values for the same 100 pages that resulted from the 10 iterations run.  


<h2 style="color:darkgreen"> HW 9.3 Implementation </h2>

In [1]:
## Code goes here

In [2]:
## Drivers & Runners

In [3]:
## Run Scripts, S3 Sync

 <h2 style="color:darkgreen">  HW 9.3 Analysis </h2>




<br><br>

<h2 style="color:darkblue"> HW 9.4: Topic-specific PageRank implementation using MRJob </h2>

Modify your PageRank implementation to produce a topic specific PageRank implementation, as described in:

http://www-cs-students.stanford.edu/~taherh/papers/topic-sensitive-pagerank.pdf

Note in this article that there is a special caveat to ensure that the transition matrix is irreducible.   
This caveat lies in footnote 3 on page 3:
```
	A minor caveat: to ensure that M is irreducible when p
	contains any 0 entries, nodes not reachable from nonzero
	nodes in p should be removed. In practice this is not problematic.
```
and must be adhered to for convergence to be guaranteed.   

Run topic specific PageRank on the following randomly generated network of 100 nodes:

> s3://ucb-mids-mls-networks/randNet.txt (also available on Dropbox)

which are organized into ten topics, as described in the file:

> s3://ucb-mids-mls-networks/randNet_topics.txt  (also available on Dropbox)

Since there are 10 topics, your result should be 11 PageRank vectors (one for the vanilla PageRank implementation in 9.1, and one for each topic with the topic specific implementation). Print out the top ten ranking nodes and their topics for each of the 11 versions, and comment on your result. Assume a teleportation factor of 0.15 in all your analyses.

One final and important comment here:  please consider the requirements for irreducibility with topic-specific PageRank. In particular, the literature ensures irreducibility by requiring that nodes not reachable from in-topic nodes be removed from the network.

This is not a small task, especially as it it must be performed separately for each of the (10) topics.

So, instead of using this method for irreducibility, please comment on why the literature's method is difficult to implement, and what what extra computation it will require.   

Then for your code, please use the alternative, non-uniform damping vector:

```
vji = beta*(1/|Tj|); if node i lies in topic Tj

vji = (1-beta)*(1/(N - |Tj|)); if node i lies outside of topic Tj
```
for beta in (0,1) close to 1. 

With this approach, you will not have to delete any nodes. If beta > 0.5, PageRank is topic-sensitive, and if beta < 0.5, the PageRank is anti-topic-sensitive. For any value of beta irreducibility should hold, so please try beta=0.99, and perhaps some other values locally, on the smaller networks.

<h2 style="color:darkgreen"> HW 9.4 Implementation </h2>

In [1]:
## Code goes here

In [2]:
## Drivers & Runners

In [3]:
## Run Scripts, S3 Sync

<h2 style="color:darkgreen">  HW 9.4 Analysis </h2>




<br><br>

<center><div class='jumbotron'><h3 style='color:darkblue'>---------  OPTIONAL QUESTIONS SECTION --------</h3></div></center>

<h2 style="color:darkblue"> HW 9.5: (OPTIONAL) Applying topic-specific PageRank to Wikipedia</h2>

Here you will apply your topic-specific PageRank implementation to Wikipedia, defining topics (very arbitrarily) for each page by the length (number of characters) of the name of the article mod 10, so that there are 10 topics. 

* Once again, print out the top ten ranking nodes and their topics for each of the 11 versions, and comment on your result. Assume a teleportation factor of 0.15 in all your analyses. Run for 10 iterations.
* Plot the pagerank values for the top 100 pages resulting from the 5 iterations run in HW 9.3. 
* Then plot the pagerank values for the same 100 pages that result from the topic specific pagerank after 10 iterations run. 
* Comment on your findings. 

<h2 style="color:darkgreen"> HW 9.5 Implementation </h2>

In [1]:
## Code goes here

In [2]:
## Drivers & Runners

In [3]:
## Run Scripts, S3 Sync

<h2 style="color:darkgreen">  HW 9.5 Analysis </h2>




<br><br>

<h2 style="color:darkblue"> HW 9.6:  (OPTIONAL) TextRank</h2>

* What is TextRank? Describe the main steps in the algorithm. Why does TextRank work?
* Implement TextRank in MrJob for keyword phrases (not just unigrams) extraction using co-occurrence based similarity measure with with sizes of N = 2 and 3. And evaluate your code using the following example using precision, recall, and FBeta (Beta=1):
```
"Compatibility of systems of linear constraints over the set of natural numbers
Criteria of compatibility of a system of linear Diophantine equations, strict 
inequations, and nonstrict inequations are considered. Upper bounds for
components of a minimal set of solutions and algorithms of construction of 
minimal generating sets of solutions for all types of systems are given. 
These criteria and the corresponding algorithms for constructing a minimal 
supporting set of solutions can be used in solving all the considered types of 
systems and systems of mixed types." 
```
* The extracted keywords should in the following set:
```
linear constraints, linear diophantine equations, natural numbers, non-strict inequations, strict inequations, upper bounds
```

<h2 style="color:darkgreen"> HW 9.6 Implementation </h2>

In [1]:
## Code goes here

In [2]:
## Drivers & Runners

In [3]:
## Run Scripts, S3 Sync

<h2 style="color:darkgreen">  HW 9.6 Analysis </h2>




<br><br>

<center><div class='jumbotron'><h2 style='color:green'>-------  END OF HWK 9 --------</h2></div></center>